In [1]:
from sys import path
import os
import glob
path.append('/home/rui/blacksid/mew-p')
DATA_PATH = '/home/rui/blacksid/data/dockfuture'
import numpy as np
import pytz
from datetime import datetime
import cProfile
import matplotlib.pyplot as plt
from mewp.simulate.wrapper import TrendAlgoWrapper
from mewp.simulate.runner import SingleRunner
import pandas as pd
from mewp.util.clock import Clock
from mewp.util.candle import Candle
from mewp.util.candle import CandleCreator
from mewp.util.candle import CandleAnalysis

In [2]:
os.chdir('/home/rui/blacksid/data/stock/daily')

In [3]:
product = 'C'

In [4]:
contract_file_list = glob.glob("{}1*.csv".format(product))
CONTRACT_LIST = [x.split('.')[0] for x in contract_file_list]
CONTRACT_DF = {}
for contract in CONTRACT_LIST:
    temp = pd.read_csv(contract + '.csv')
    CONTRACT_DF[contract] = temp

In [5]:
dates = [str(x).split(' ')[0] for x in pd.date_range('2010-01-01','2016-06-01').tolist()]

In [6]:
def get_contract_info_at_day(contract, date):
    df = CONTRACT_DF[contract]
    index = (df.datetime == date)
    if sum(index) == 0:
        return np.nan
    elif sum(index) == 1:
        return df[index]
    else:
        print '********warning: more than two rows for a day'
        return np.nan

In [7]:
type(np.nan)

float

In [8]:
def is_trade_day(date):
    for contract in CONTRACT_LIST:
        if not type(get_contract_info_at_day(contract, date))==float:
            return True
    return False

In [9]:
dates = [str(x).split(' ')[0] for x in pd.date_range('2010-01-01','2016-09-01').tolist()]
ALL_DATES = []
for date in dates:
    if is_trade_day(date):
        ALL_DATES.append(date)

In [10]:
# Brief: input a date, output the main contract of that date.
def get_contract_score(date):
    date_contract_list = []
    contract_volume_list = []
    contract_position_list = []
    for contract in CONTRACT_LIST:
        df = get_contract_info_at_day(contract, date)
        if not type(df)==float:
            date_contract_list.append(contract)
            contract_volume_list.append(float(df.TotalVolumeTraded))
            contract_position_list.append(float(df.OpenInterest))
    if len(date_contract_list) == 0:
        return
    max_volume = max(contract_volume_list)
    max_position = max(contract_position_list)
    contract_volume_list = map(lambda x: x/max_volume, contract_volume_list)
    contract_position_list = map(lambda x: x/max_position, contract_position_list)
    relative_score_list = [(i+j)/2 for i,j in zip(contract_volume_list, contract_position_list)]
    score = dict(zip(date_contract_list, relative_score_list))
    return score

In [11]:
temp = get_contract_score('2016-01-04')

In [12]:
temp

{'C1601': 0.005655361366160212,
 'C1603': 0.00026812386091269556,
 'C1605': 1.0,
 'C1607': 0.000677070366243268,
 'C1609': 0.49234260796338364,
 'C1611': 0.0007975741580471273}

In [13]:
def get_main_contract(date_list, window):
    trade_day_list = []
    main_contract_list = []
    score_queue = []
    for date in date_list:
        contracts = None
        score = get_contract_score(date)
        if type(score) != dict:
            continue
        score_queue.append(score)
        if len(score_queue) > window:
            score_queue.pop(0)
        i = 0
        for score in reversed(score_queue):
            if contracts == None:
                contracts = set(score.keys())
            elif len(contracts.intersection(score.keys())) > 0:
                contracts = contracts.intersection(score.keys())
            else:
                break
            i+=1
        ma_score = {}
        for contract in contracts:
            ma = 0
            for k in range(1,i+1):
                ma+=score_queue[-k][contract]
            ma_score[contract] = ma
        main_contract = max(ma_score, key=ma_score.get)
        main_contract_list.append(main_contract)
        trade_day_list.append(date)
    return zip(trade_day_list, main_contract_list)

In [14]:
# main contract:
result = get_main_contract(ALL_DATES,10)

In [15]:
result

[('2010-01-04', 'C1009'),
 ('2010-01-05', 'C1009'),
 ('2010-01-06', 'C1009'),
 ('2010-01-07', 'C1009'),
 ('2010-01-08', 'C1009'),
 ('2010-01-11', 'C1009'),
 ('2010-01-12', 'C1009'),
 ('2010-01-13', 'C1009'),
 ('2010-01-14', 'C1009'),
 ('2010-01-15', 'C1009'),
 ('2010-01-18', 'C1009'),
 ('2010-01-19', 'C1009'),
 ('2010-01-20', 'C1009'),
 ('2010-01-21', 'C1009'),
 ('2010-01-22', 'C1009'),
 ('2010-01-25', 'C1009'),
 ('2010-01-26', 'C1009'),
 ('2010-01-27', 'C1009'),
 ('2010-01-28', 'C1009'),
 ('2010-01-29', 'C1009'),
 ('2010-02-01', 'C1009'),
 ('2010-02-02', 'C1009'),
 ('2010-02-03', 'C1009'),
 ('2010-02-04', 'C1009'),
 ('2010-02-05', 'C1009'),
 ('2010-02-08', 'C1009'),
 ('2010-02-09', 'C1009'),
 ('2010-02-10', 'C1009'),
 ('2010-02-11', 'C1009'),
 ('2010-02-12', 'C1009'),
 ('2010-02-22', 'C1009'),
 ('2010-02-23', 'C1009'),
 ('2010-02-24', 'C1009'),
 ('2010-02-25', 'C1009'),
 ('2010-02-26', 'C1009'),
 ('2010-03-01', 'C1009'),
 ('2010-03-02', 'C1009'),
 ('2010-03-03', 'C1009'),
 ('2010-03-0

In [18]:
a = pd.read_csv('C1009'+'.csv',index_col =0).index

In [20]:
a

Index([u'2010-01-04', u'2010-01-05', u'2010-01-06', u'2010-01-07',
       u'2010-01-08', u'2010-01-11', u'2010-01-12', u'2010-01-13',
       u'2010-01-14', u'2010-01-15',
       ...
       u'2010-09-01', u'2010-09-02', u'2010-09-03', u'2010-09-06',
       u'2010-09-07', u'2010-09-08', u'2010-09-09', u'2010-09-10',
       u'2010-09-13', u'2010-09-14'],
      dtype='object', name=u'datetime', length=172)

In [22]:
a == '2010-01-08'

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [28]:
pre_main_contract = None
pre_df = None
price_adjust = 0
for date, contract in reversed(result):
    today = datetime.strptime(date,'%Y-%m-%d')
    if pre_main_contract==None:
        pass
    elif pre_main_contract == contract:
        pass
    else:
        pre = get_contract_info_at_day(pre_main_contract, date)
        cur = get_contract_info_at_day(contract, date)
        price_adjust += float(pre.ClosingPx) - float(cur.ClosingPx)
    temp_df = pd.read_csv(contract+'.csv',index_col =0)
    bool_index = temp_df.index == date
    temp_df = temp_df[bool_index]
    temp_df['adjust'] = price_adjust
    pre_df = pd.concat([temp_df,pre_df])
    pre_main_contract = contract

In [29]:
pre_df

,OpeningPx,ClosingPx,HighPx,LowPx,TotalTurnover,TotalVolumeTraded,SettlPx,PrevSettlPx,OpenInterest,BasisSpread,LimitUp,LimitDown,adjust
datetime,,,,,,,,,,,,,
2010-01-04,1878.0,1892.0,1900.0,1875.0,3.334698e+09,176352.0,1890.0,1870.0,217210.0,NaN,NaN,NaN,-568.0
2010-01-05,1893.0,1891.0,1904.0,1889.0,1.962358e+09,103552.0,1895.0,1890.0,226796.0,NaN,NaN,NaN,-568.0
2010-01-06,1895.0,1908.0,1918.0,1890.0,3.056239e+09,160396.0,1905.0,1895.0,241304.0,NaN,NaN,NaN,-568.0
2010-01-07,1910.0,1898.0,1921.0,1881.0,3.094342e+09,162554.0,1903.0,1905.0,228008.0,NaN,NaN,NaN,-568.0
2010-01-08,1898.0,1925.0,1927.0,1895.0,1.159226e+10,605112.0,1915.0,1903.0,298488.0,NaN,NaN,NaN,-568.0
2010-01-11,1932.0,1921.0,1936.0,1918.0,4.668005e+09,242470.0,1925.0,1915.0,305364.0,NaN,NaN,NaN,-568.0
2010-01-12,1915.0,1908.0,1922.0,1905.0,3.741112e+09,195492.0,1913.0,1925.0,288226.0,NaN,NaN,NaN,-568.0
2010-01-13,1885.0,1887.0,1896.0,1879.0,4.102040e+09,217122.0,1889.0,1913.0,230980.0,NaN,NaN,NaN,-568.0
2010-01-14,1898.0,1896.0,1903.0,1892.0,2.056089e+09,108336.0,1897.0,1889.0,225416.0,NaN,NaN,NaN,-568.0


In [26]:
date

'2016-08-24'

In [65]:
def save_adjusted_continuous_price(date_list, window):
    main_contract_list = get_main_contract(date_list, window)
    pre_main_contract = None
    datetime_ = []
    OpeningPx_ = []
    ClosingPx_ = []
    HighPx_ = []
    LowPx_ = []
    TotalVolumeTraded_ = []
    SettlPx_ = []
    PrevSettlPx_ = []
    OpenInterest_ = []
    result = {}
    for date, contract in reversed(main_contract_list):
        cur = get_contract_info_at_day(contract, date)
        if pre_main_contract==None:
            price_adjust = 0
        elif pre_main_contract == contract:
            pass
        else:
            pre = get_contract_info_at_day(pre_main_contract, date)
            price_adjust = float(pre.ClosingPx) - float(cur.ClosingPx)
        datetime_.append(date)
        OpeningPx_.append(float(cur.OpeningPx) + price_adjust)
        ClosingPx_.append(float(cur.ClosingPx) + price_adjust)
        HighPx_.append(float(cur.HighPx) + price_adjust)
        LowPx_.append(float(cur.LowPx) + price_adjust)
        TotalVolumeTraded_.append(float(cur.TotalVolumeTraded))
        SettlPx_.append(float(cur.SettlPx) + price_adjust)
        PrevSettlPx_.append(float(cur.PrevSettlPx) + price_adjust)
        OpenInterest_.append(float(cur.OpenInterest))
        pre_main_contract = contract
    
    datetime_.reverse()
    OpeningPx_.reverse()
    ClosingPx_.reverse()
    HighPx_.reverse()
    LowPx_.reverse()
    TotalVolumeTraded_.reverse()
    SettlPx_.reverse()
    PrevSettlPx_.reverse()
    OpenInterest_.reverse()
    result['datetime'] = datetime_
    result['OpeningPx'] = OpeningPx_
    result['ClosingPx'] = ClosingPx_
    result['HighPx'] = HighPx_
    result['LowPx'] = LowPx_
    result['TotalVolumeTraded'] = TotalVolumeTraded_
    result['SettlPx'] = SettlPx_
    result['PrevSettlPx'] = PrevSettlPx_
    result['OpenInterest'] = OpenInterest_
    df = pd.DataFrame(result)
    return df
        

In [66]:
df = save_adjusted_continuous_price(ALL_DATES, 10)

In [67]:
df

,ClosingPx,HighPx,LowPx,OpenInterest,OpeningPx,PrevSettlPx,SettlPx,TotalVolumeTraded,datetime
0,60050.0,60730.0,59840.0,161040.0,60170.0,59710.0,60390.0,285600.0,2010-01-04
1,60610.0,60690.0,59920.0,160152.0,60470.0,60390.0,60380.0,270286.0,2010-01-05
2,61650.0,61960.0,60530.0,172100.0,60680.0,60380.0,61420.0,317130.0,2010-01-06
3,61190.0,63470.0,60530.0,159808.0,62090.0,61420.0,61880.0,585286.0,2010-01-07
4,60710.0,61480.0,59700.0,155876.0,61220.0,61880.0,60520.0,533576.0,2010-01-08
5,62190.0,62540.0,61550.0,156532.0,61570.0,60520.0,62010.0,313530.0,2010-01-11
6,61720.0,61870.0,61070.0,151668.0,61540.0,62010.0,61500.0,269398.0,2010-01-12
7,59790.0,60500.0,58790.0,157426.0,59520.0,61500.0,59880.0,379902.0,2010-01-13
8,60770.0,61260.0,60620.0,150502.0,61030.0,59880.0,60900.0,180786.0,2010-01-14
9,60530.0,60980.0,60190.0,148232.0,60730.0,60900.0,60590.0,157256.0,2010-01-15


In [71]:
df.to_csv('/home/rui/blacksid/data/adjusted_daily/{}0.csv'.format(product))